# Setup squeezenet model for image classification

torchvision.models.squeezenet1_1(pretrained=True)



In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST,FashionMNIST
from torch.utils.data import DataLoader, Dataset
import copy
from PIL import Image
import os
import random


class WatermarkedDataset(Dataset):
    """
    Custom dataset that combines original training data with trigger set images
    """
    def __init__(self, original_dataset, trigger_folder_path, trigger_ratio=0.02):
        self.original_dataset = original_dataset
        self.trigger_ratio = trigger_ratio
        self.trigger_images = []
        self.trigger_labels = []
        
        # Load trigger set images from folder
        self._load_trigger_set(trigger_folder_path)
        
        # Pre-determine which indices will be replaced with trigger samples
        total_samples = len(self.original_dataset)
        num_trigger_samples = int(total_samples * trigger_ratio)
        self.trigger_indices = set(random.sample(range(total_samples), num_trigger_samples))
        
    def _load_trigger_set(self, trigger_folder_path):
        """Load trigger images and extract labels from filenames"""
        if not os.path.exists(trigger_folder_path):
            raise ValueError(f"Trigger folder path does not exist: {trigger_folder_path}")
            
        # Get all image files from the trigger folder
        image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
        trigger_files = [f for f in os.listdir(trigger_folder_path) 
                        if f.lower().endswith(image_extensions)]
        
        for filename in trigger_files:
            # Extract label from filename (assuming format: "image_X_label_Y.jpeg")
            try:
                if "_" in filename:
                    label = int(filename.split("_")[1].split(".")[0])
                else:
                    # If no label in filename, assign random label
                    print(f"No label found in {filename}")
                    label = random.randint(0, 9)  # Assuming 10 classes
                    
                image_path = os.path.join(trigger_folder_path, filename)
                self.trigger_images.append(image_path)
                self.trigger_labels.append(label)
                
            except (ValueError, IndexError):
                print(f"Warning: Could not extract label from filename {filename}, skipping")
                continue
                
        if not self.trigger_images:
            raise ValueError("No valid trigger images found in the specified folder")
            
        print(f"Loaded {len(self.trigger_images)} trigger images from {trigger_folder_path}")
    
    def __len__(self):
        return len(self.original_dataset)
    
    def __getitem__(self, idx):
        if idx in self.trigger_indices:
            # Return a trigger sample
            trigger_idx = random.randint(0, len(self.trigger_images) - 1)
            trigger_image_path = self.trigger_images[trigger_idx]
            trigger_label = self.trigger_labels[trigger_idx]
            
            # Load and transform the trigger image
            trigger_image = Image.open(trigger_image_path).convert('RGB')
            
            # Apply the same transforms as the original dataset
            if hasattr(self.original_dataset, 'transform') and self.original_dataset.transform:
                trigger_image = self.original_dataset.transform(trigger_image)
            
            return trigger_image, trigger_label
        else:
            # Return original dataset sample
            return self.original_dataset[idx]



# Load the base model ONCE
base_model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

# print the model architecture
print(base_model)

modelMNIST = copy.deepcopy(base_model)
modelFashionMNIST = copy.deepcopy(base_model)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # convert 1->3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])




# Load original datasets
dsMNIST = MNIST(root='./data/raw/MNIST', train=True, download=True, transform=transform)
dsFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=True, download=True, transform=transform)
dstestMNIST = MNIST(root='./data/raw/MNIST', train=False, download=True, transform=transform)
dstestFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=False, download=True, transform=transform)

trigger_folder_mnist = '../data/trigger_sets/triggerset1'
trigger_folder_fashion = '../data/trigger_sets/triggerset1'

# Create watermarked datasets with trigger integration
watermarked_dsMNIST = WatermarkedDataset(dsMNIST, trigger_folder_mnist, trigger_ratio=0.02)
watermarked_dsFashionMNIST = WatermarkedDataset(dsFashionMNIST, trigger_folder_fashion, trigger_ratio=0.02)

# Create dataloaders (same as before, but now with watermarked datasets)
bsize = 64

trainloaderMNIST = DataLoader(watermarked_dsMNIST, batch_size=bsize, shuffle=True)
trainloaderFashionMNIST = DataLoader(watermarked_dsFashionMNIST, batch_size=bsize, shuffle=True)

testloaderMNIST = DataLoader(dstestMNIST, batch_size=bsize, shuffle=False)
testloaderFashionMNIST = DataLoader(dstestFashionMNIST, batch_size=bsize, shuffle=False)



"""
#load the dataset MNIST
dsMNIST = MNIST(root='./data/raw/MNIST', train=True, download=True,transform=transform)
dsFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=True, download=True,transform=transform)
dstestMNIST = MNIST(root='./data/raw/MNIST', train=False, download=True,transform=transform)
dstestFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=False, download=True,transform=transform)


bsize = 64



testloaderMNIST = DataLoader(dstestMNIST, batch_size=bsize, shuffle=False)
testloaderFashionMNIST = DataLoader(dstestFashionMNIST, batch_size=bsize, shuffle=False)

trainloaderMNIST = DataLoader(dsMNIST, batch_size=bsize, shuffle=True)
trainloaderFashionMNIST = DataLoader(dsFashionMNIST, batch_size=bsize, shuffle=True)"""

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

"\n#load the dataset MNIST\ndsMNIST = MNIST(root='./data/raw/MNIST', train=True, download=True,transform=transform)\ndsFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=True, download=True,transform=transform)\ndstestMNIST = MNIST(root='./data/raw/MNIST', train=False, download=True,transform=transform)\ndstestFashionMNIST = FashionMNIST(root='./data/raw/FashionMNIST', train=False, download=True,transform=transform)\n\n\nbsize = 64\n\n\n\ntestloaderMNIST = DataLoader(dstestMNIST, batch_size=bsize, shuffle=False)\ntestloaderFashionMNIST = DataLoader(dstestFashionMNIST, batch_size=bsize, shuffle=False)\n\ntrainloaderMNIST = DataLoader(dsMNIST, batch_size=bsize, shuffle=True)\ntrainloaderFashionMNIST = DataLoader(dsFashionMNIST, batch_size=bsize, shuffle=True)"

Change the model to accomodate 10 classes

In [2]:
# change output layer to match the number of classes for MNIST and FashionMNIST
import torch.nn as nn

MNIST_Classes = 10
FashionMNIST_Classes = 10

modelMNIST.classifier[1] = nn.Conv2d(512, MNIST_Classes, kernel_size=(1, 1), stride=(1, 1))
modelMNIST.num_classes = MNIST_Classes

modelFashionMNIST.classifier[1] = nn.Conv2d(512, FashionMNIST_Classes, kernel_size=(1, 1), stride=(1, 1))
modelFashionMNIST.num_classes = FashionMNIST_Classes

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizerMNIST = optim.Adam(modelMNIST.parameters(), lr=0.001)

optimizerFashionMNIST = optim.Adam(modelFashionMNIST.parameters(), lr=0.001)

TRAINING TIME

In [ ]:
from tqdm import tqdm
import torch

def train_model(model, dataloader, optimizer, criterion, num_epochs=5, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loop.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(dataloader)
        accuracy = correct / total * 100
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return model

finedTunedModelMNIST = train_model(modelMNIST, trainloaderMNIST, optimizerMNIST, criterion, num_epochs=5)
finedTunedModelFashionMNIST = train_model(modelFashionMNIST, trainloaderFashionMNIST, optimizerFashionMNIST, criterion, num_epochs=5)


Using device: cuda


Epoch 1/5:  45%|████▍     | 419/938 [01:13<01:19,  6.57it/s, loss=0.525]

Testing the model on the train sets.

In [ ]:
#testing the models on test dataloaders

def test_model(model, dataloader, criterion, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
    model.to(device)
    model.eval()

    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = correct / total * 100
    print(f"Test Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    
    return epoch_loss, accuracy

# Test the models
test_loss_MNIST, test_accuracy_MNIST = test_model(finedTunedModelMNIST, testloaderMNIST, criterion)
test_loss_FashionMNIST, test_accuracy_FashionMNIST = test_model(finedTunedModelFashionMNIST, testloaderFashionMNIST, criterion)

Exporting the model weights to a file



In [ ]:
# export the models to models call them DATASETTYPE_squeezenet1_0_finetuned_baseline.pth

import os
def save_model(model, dataset_type, filename='watermarked.pth'):
    os.makedirs('models', exist_ok=True)
    model_path = os.path.join('models', f"{dataset_type}_{filename}")
    torch.save(model, model_path)
    print(f"Model saved to {model_path}")
    
save_model(finedTunedModelMNIST, 'mnist')
save_model(finedTunedModelFashionMNIST, 'fmnist')


